# Import Module

In [29]:
import numpy as np
import pandas as pd

# Load Data

In [30]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
train_test = pd.concat([train, test], axis=0)
train_test["date"] = pd.to_datetime(train_test["date"], format="%Y%m%d")
train_test = train_test.reset_index(drop=True)

print(train_test.area.unique())

In [31]:
weather = pd.read_csv("./scraped_weather.csv")
weather = weather.drop(columns=["max_temp_time", "min_temp_time", "sun_time"])
for col in weather.columns:
    if col not in ["date", "area"]:
        weather[col] = weather[col].replace("--", 0)
        weather[col] = weather[col].replace("]", 0)
        weather[col] = weather[col].astype(float)
print(weather.area.unique())

/home/higuchi-lab/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [32]:
train_test

,kind,date,amount,mode_price,area
0,だいこん,2005-11-07,201445.0,735.0,千葉
1,だいこん,2005-11-08,189660.0,840.0,千葉_各地_青森
2,だいこん,2005-11-10,218166.0,735.0,千葉_各地_青森
3,だいこん,2005-11-11,182624.0,682.5,千葉_青森
4,だいこん,2005-11-12,220691.0,682.5,千葉_青森
...,...,...,...,...,...
98097,ミニトマト,2022-05-26,NaN,NaN,各地_愛知
98098,ミニトマト,2022-05-27,NaN,NaN,各地_愛知
98099,ミニトマト,2022-05-28,NaN,NaN,各地_愛知
98100,ミニトマト,2022-05-30,NaN,NaN,各地_愛知


In [33]:
weather

,date,mean_temp,max_temp,min_temp,sum_rain,mean_humid,area
0,2004-11-06,13.1,20.1,8.1,5.0,77.0,青森
1,2004-11-07,9.6,12.5,5.4,0.0,63.0,青森
2,2004-11-08,9.0,15.5,3.2,0.0,72.0,青森
3,2004-11-09,12.2,17.7,5.7,1.5,85.0,青森
4,2004-11-10,11.6,16.4,6.9,0.0,89.0,青森
...,...,...,...,...,...,...,...
205307,2022-05-27,21.0,24.6,19.2,59.5,91.0,東京
205308,2022-05-28,22.4,27.7,16.8,0.0,62.0,東京
205309,2022-05-29,24.1,31.2,16.6,0.0,53.0,東京
205310,2022-05-30,22.3,27.8,18.8,0.0,70.0,東京


# 1. Map train(test).csv's Areas To weather.csv's Areas

In [34]:
area_pairs = train_test['area'].unique()
yasai_areas = set()

for area_pair in area_pairs:
    areas = area_pair.split('_')
    yasai_areas |= set(areas)

yasai_areas

{'アメリカ',
 'カナダ',
 'トンガ',
 'ニュージーランド',
 'メキシコ',
 '中国',
 '佐賀',
 '兵庫',
 '北海道',
 '千葉',
 '各地',
 '和歌山',
 '埼玉',
 '宮城',
 '宮崎',
 '山形',
 '山梨',
 '岩手',
 '徳島',
 '愛媛',
 '愛知',
 '新潟',
 '東京',
 '栃木',
 '沖縄',
 '熊本',
 '神奈川',
 '福岡',
 '福島',
 '秋田',
 '群馬',
 '茨城',
 '長崎',
 '長野',
 '青森',
 '静岡',
 '香川',
 '高知',
 '鹿児島'}

In [35]:
wea_areas = weather['area'].unique()
wea_areas

array(['青森', '盛岡', '秋田', '仙台', '山形', '福島', '水戸', '宇都宮', '前橋', '熊谷', '甲府',
       '横浜', '千葉', '浜松', '長野', '新潟', '名古屋', '高松', '徳島', '高知', '松山', '那覇',
       '鹿児島', '宮崎', '長崎', '熊本', '福岡', '佐賀', '和歌山', '神戸', '帯広', '東京'],
      dtype=object)

In [36]:
area_map = {}
update_area_map = {
    '岩手':'盛岡','宮城':'仙台','静岡':'浜松','沖縄':'那覇','神奈川':'横浜','愛知':'名古屋','茨城':'水戸','北海道':'帯広','各地':'全国',
    '兵庫':'神戸','香川':'高松','埼玉':'熊谷','国内':'全国','山梨':'甲府','栃木':'宇都宮','群馬':'前橋','愛媛':'松山'
}

In [37]:
for yasai_area in yasai_areas:
    if yasai_area not in wea_areas and yasai_area not in update_area_map:
        # yasai_area not in wea_areas => not duplicated
        # yasai_area not in update_area_map => not mapped yet

        area_map[yasai_area] = '全国'
    else:
        # duplicated
        area_map[yasai_area] = yasai_area
area_map = {**area_map, **update_area_map}
area_map

{'和歌山': '和歌山',
 '愛知': '名古屋',
 '香川': '高松',
 '中国': '全国',
 '宮崎': '宮崎',
 '徳島': '徳島',
 '北海道': '帯広',
 'ニュージーランド': '全国',
 '静岡': '浜松',
 '佐賀': '佐賀',
 '栃木': '宇都宮',
 '兵庫': '神戸',
 '熊本': '熊本',
 '福島': '福島',
 '秋田': '秋田',
 'トンガ': '全国',
 '茨城': '水戸',
 '鹿児島': '鹿児島',
 '山形': '山形',
 'カナダ': '全国',
 '長野': '長野',
 '山梨': '甲府',
 '千葉': '千葉',
 '沖縄': '那覇',
 '岩手': '盛岡',
 '福岡': '福岡',
 '新潟': '新潟',
 'メキシコ': '全国',
 '東京': '東京',
 '各地': '全国',
 '高知': '高知',
 'アメリカ': '全国',
 '長崎': '長崎',
 '埼玉': '熊谷',
 '神奈川': '横浜',
 '青森': '青森',
 '群馬': '前橋',
 '宮城': '仙台',
 '愛媛': '松山',
 '国内': '全国'}

In [38]:
train_test['area'] = train_test['area'].apply(lambda x: '_'.join([area_map[i] for i in x.split('_')]))
train_test

,kind,date,amount,mode_price,area
0,だいこん,2005-11-07,201445.0,735.0,千葉
1,だいこん,2005-11-08,189660.0,840.0,千葉_全国_青森
2,だいこん,2005-11-10,218166.0,735.0,千葉_全国_青森
3,だいこん,2005-11-11,182624.0,682.5,千葉_青森
4,だいこん,2005-11-12,220691.0,682.5,千葉_青森
...,...,...,...,...,...
98097,ミニトマト,2022-05-26,NaN,NaN,全国_名古屋
98098,ミニトマト,2022-05-27,NaN,NaN,全国_名古屋
98099,ミニトマト,2022-05-28,NaN,NaN,全国_名古屋
98100,ミニトマト,2022-05-30,NaN,NaN,全国_名古屋


In [39]:
train_test.to_csv('mapped_train_test.csv', index=False)

# 2. Adjust weather.csv for Unknown Areas(e.g. 全国, 千葉_全国_横浜, ...)

## Creation of key (全国) to absorb foreign countries, 各地, etc.

In [40]:
tmp = weather.copy()
dates = tmp.date.unique()
tmp = tmp.drop(columns=["area"])

zenkoku = {}
for group, series in tmp.groupby("date"):
    zenkoku[group] = series.mean().values
zenkoku = pd.DataFrame(zenkoku, index=tmp.columns[1:]).T
zenkoku["date"] = dates
zenkoku["area"] = "全国"

weather = pd.concat([weather, zenkoku], axis=0)

## Adjust weather.csv

In [41]:
area_pairs = train_test['area'].unique()
area_pairs

array(['千葉', '千葉_全国_青森', '千葉_青森', '千葉_全国', '千葉_横浜', '千葉_全国_横浜', '横浜',
       '千葉_全国_水戸', '千葉_水戸', '青森', '帯広_青森', '帯広_全国_青森', '帯広_全国',
       '帯広_千葉_全国', '全国_横浜', '全国_青森', '全国', '帯広', '千葉_熊谷', '千葉_全国_熊谷',
       '全国_熊谷', '熊谷', '帯広_千葉', '千葉_徳島', '千葉_全国_徳島', '徳島', '全国_徳島',
       '全国_全国_徳島', '全国_帯広', '全国_徳島_浜松', '徳島_浜松', '水戸', '全国_水戸',
       '全国_水戸_青森', '水戸_青森', '佐賀_水戸', '佐賀_全国_水戸', '千葉_全国_名古屋', '千葉_名古屋',
       '名古屋', '全国_名古屋_横浜', '盛岡_前橋', '全国_盛岡_前橋', '帯広_全国_前橋', '全国_前橋', '前橋',
       '千葉_全国_前橋', '千葉_前橋', '全国_名古屋', '名古屋_横浜', '全国_前橋_水戸', '全国_水戸_高松',
       '全国_浜松_高松', '全国_高松', '全国_水戸_長野', '水戸_長野', '長野', '前橋_長野',
       '全国_前橋_長野', '全国_長野', '浜松_高松', '全国_水戸_浜松', '神戸_全国_水戸', '神戸_全国_浜松',
       '全国_浜松', '神戸_水戸', '神戸_全国', '神戸_浜松', '水戸_浜松', '浜松', '全国_水戸_長崎',
       '前橋_水戸', '熊谷_東京', '全国_熊谷_東京', '全国_東京', '東京', '全国_熊谷_前橋', '熊谷_前橋',
       '全国_宇都宮', '宇都宮_前橋', '宇都宮', '宇都宮_水戸', '全国_宇都宮_前橋', '全国_全国_青森',
       '全国_熊谷_青森', '熊谷_青森', '全国_秋田_青森', '秋田_青森', '全国_秋田', '千葉_宇都宮',
       '千葉_全国_宇都宮', '全国_熊谷_名古

In [42]:
weather

,date,mean_temp,max_temp,min_temp,sum_rain,mean_humid,area
0,2004-11-06,13.100000,20.100000,8.100000,5.000000,77.00000,青森
1,2004-11-07,9.600000,12.500000,5.400000,0.000000,63.00000,青森
2,2004-11-08,9.000000,15.500000,3.200000,0.000000,72.00000,青森
3,2004-11-09,12.200000,17.700000,5.700000,1.500000,85.00000,青森
4,2004-11-10,11.600000,16.400000,6.900000,0.000000,89.00000,青森
...,...,...,...,...,...,...,...
2022-05-27,2022-05-27,20.700000,25.325000,17.053125,23.765625,76.62500,全国
2022-05-28,2022-05-28,21.309375,26.203125,16.612500,1.796875,60.59375,全国
2022-05-29,2022-05-29,22.590625,29.006250,16.493750,1.125000,56.75000,全国
2022-05-30,2022-05-30,20.725000,25.856250,16.518750,4.484375,72.31250,全国


In [43]:
target_cols = [i for i in weather.columns if i not in ['area', 'date']]
dates = weather.date.unique()
weather = weather.drop(columns="date")
adjusted_weather = []

for area_pair in area_pairs:
    areas = area_pair.split("_")

    if len(areas) > 0:
        area = areas[0]
        base = weather[weather["area"] == area]
        base = base[target_cols].reset_index(drop=True)

        for area in areas[1:]:
            tmp = weather[weather["area"] == area]
            tmp = tmp[target_cols].reset_index(drop=True)
            base = base.add(tmp)
            # Add value every element of the matrix

        base /= len(areas)
        # Mean
        base["areas"] = area_pair
        base["dates"] = dates
        adjusted_weather.append(base)
adjusted_weather = pd.concat(adjusted_weather, axis=0)

In [44]:
adjusted_weather.to_csv('mapped_adjusted_weather.csv', index=False)